# 데이터가 너무 많아 학습 오래걸리므로 전체 데이터의 subset 생성 및, train val split

## multi label stf k fold를 진행하기 위한 df 생성

In [110]:
import json
import os
import numpy as np
import pandas as pd

from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold

In [2]:
## 간단한 EDA

In [7]:
# %cd bootcamp/dataset
# !ls

/opt/ml/bootcamp/dataset
4th_run_124.yaml  labels124  train_124.txt  val_124.txt
class_124.csv	  train      val


In [14]:
# FP를 줄이기 위해 label이 없는 데이터도 존재
all_train_num = len(os.listdir('train/images'))
yes_label_train_num = len(os.listdir('labels124')) 
print(f'전체 이미지 개수: {all_train_num}\nlabel이 존재하는 이미지 개수:{yes_label_train_num}')

전체 이미지 개수: 81162
label이 존재하는 이미지 개수:73566


In [489]:
no_label_img_list = [] # label이 존재하지 않는 image list 
train_img_list = os.listdir('train/images')
yes_label_img_list = os.listdir('labels124')
for i in tqdm(range(len(train_img_list))):
    img_path = train_img_list[i]
    img_path_split = img_path.split('.')
    img_name = img_path_split[0] + '.' + img_path_split[1] + '.txt'
    if img_name not in yes_label_img_list:
        no_label_img_list.append(img_path)
len(no_label_img_list) == 81162 - 73566

100%|██████████| 81162/81162 [01:06<00:00, 1225.48it/s]


True

In [112]:
def get_session(class_num):
    if class_num <= 8:
        session = 0
    elif class_num <=18:
        session = 1
    elif class_num <=28:
        session = 2
    elif class_num <=38:
        session = 3
    elif class_num <=47:
        session = 4
    elif class_num <=57:
        session = 5
    elif class_num <=67:
        session = 6
    elif class_num <=73:
        session = 7
    elif class_num <=83:
        session = 8
    elif class_num <=93:
        session = 9
    elif class_num <=103:
        session = 10
    elif class_num <=113:
        session = 11
    else:
        session = 12
    
    return session

In [440]:
# 기존 train, val에 해당하는 44140 + 14713 = 58853개로 후보군 축소(0.2에 해당하는 test는 쓰이지 않아서 기존 trainset은 8만이아닌 58853)
with open('train_124.txt', 'r') as f:
    train_124 = f.readlines()
with open('val_124.txt', 'r') as f:
    val_124 = f.readlines()
train_124.extend(val_124)

In [451]:
train_58853_list = []
for path in train_124:
    train_58853_list.append(path.split('/')[-1][:-4] + 'txt')
len(train_58853_list)

58853

In [452]:
both_exist = 0
img_ann_list = os.listdir('labels124')
for img in train_58853_list:
    if img in img_ann_list:
        both_exist+=1
both_exist # -> 기존 train,val set의 이미지들은 모두 다 label txt파일이 존재

58853

In [455]:
img_ann_list = train_58853_list.copy()
label_img_df = pd.DataFrame(columns=[['img_name','object_num','box_mean_size','session', 'box_class_list']])
empty_txt_img_list = [] # txt파일 비어있는 경우
for i, img in tqdm(enumerate(img_ann_list)):
    if img == '.ipynb_checkpoints':
        continue
        
    with open(os.path.join('labels124',img), 'r') as f:
        data = f.readlines()  # class, x, y, w, h
        
    if len(data) == 0:
        empty_txt_img_list.append(img.split('.')[0] + '.' + img.split('.')[1] + '.jpg')
        continue
        
    box_size_list = []
    box_class_list = []
    for box in data:
        box_w, box_h = box.split()[-2], box.split()[-1]
        box_class = box.split()[0]
        box_size = round(float(box_w) * float(box_h), 5)
        box_size_list.append(box_size)
        box_class_list.append(box_class)
    
    img_name = img.split('.')[0] + '.' + img.split('.')[1]
    object_num = len(data)
    box_mean_size = np.mean(box_size_list)
    session = get_session(int(data[0].split()[0]))
    
    temp_df = pd.DataFrame([[img_name, object_num, box_mean_size, session, box_class_list]], columns=[['img_name','object_num','box_mean_size','session','box_class_list']])
    label_img_df = pd.concat([label_img_df, temp_df], ignore_index=True)

58853it [03:37, 270.02it/s]


In [456]:
 # 빈 txt파일이 874개, 즉 label이 존재하지 않는 data가 추가적으로 874개 존재
label_img_df.shape, len(empty_txt_img_list)

((58178, 5), 675)

In [457]:
label_img_df = label_img_df.reset_index()
label_img_df.columns = [['img_id','img_name','object_num','box_mean_size','session','box_class_list']]
label_img_df['object_num'] = label_img_df['object_num'].apply(lambda x: x.astype(int)) # decsribe 값을 보기 위해
label_img_df.head()

,img_id,img_name,object_num,box_mean_size,session,box_class_list
0,0,10_hik_204_1662356574.mov_280,3,0.001357,10,"[96, 95, 98]"
1,1,11_hik_204_1662358535.mov_1852,2,0.004645,11,"[108, 108]"
2,2,9_hik_205_1662348299.mov_3328,2,0.004400,9,"[93, 87]"
3,3,5_hik_203_1658977700.mov_2295,1,0.010570,5,[49]
4,4,4_hik_205_1658973309.mov_165,2,0.003005,4,"[42, 43]"


In [458]:
label_img_df.describe() # box_mean_size만 4등분하여 4개의 class로 분류

,img_id,object_num,box_mean_size
count,58178.000000,58178.000000,58178.000000
mean,29088.500000,2.475076,0.005458
std,16794.686317,1.156493,0.004179
min,0.000000,1.000000,0.000150
25%,14544.250000,2.000000,0.002755
50%,29088.500000,2.000000,0.004424
75%,43632.750000,3.000000,0.006970
max,58177.000000,10.000000,0.159480


In [459]:
box_mean_size_class_list = []
for i,row in label_img_df.iterrows():
    box_mean_size = row['box_mean_size']
    if box_mean_size < 0.002755:
        box_mean_size_class_list.append(0)
    elif box_mean_size < 0.004424:
        box_mean_size_class_list.append(1)
    elif box_mean_size < 0.006970:
        box_mean_size_class_list.append(2)
    else:
        box_mean_size_class_list.append(3)
label_img_df['box_mean_size_class'] = box_mean_size_class_list
label_img_df.head()

,img_id,img_name,object_num,box_mean_size,session,box_class_list,box_mean_size_class
0,0,10_hik_204_1662356574.mov_280,3,0.001357,10,"[96, 95, 98]",0
1,1,11_hik_204_1662358535.mov_1852,2,0.004645,11,"[108, 108]",2
2,2,9_hik_205_1662348299.mov_3328,2,0.004400,9,"[93, 87]",1
3,3,5_hik_203_1658977700.mov_2295,1,0.010570,5,[49],3
4,4,4_hik_205_1658973309.mov_165,2,0.003005,4,"[42, 43]",1


### mutlilabel stf k fold 진행

In [206]:
# !pip install iterative-stratification

In [460]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

cv = MultilabelStratifiedKFold(n_splits=2, shuffle=True, random_state=41)

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(label_img_df, label_img_df[['object_num', 'session', 'box_mean_size_class']])):
    
    print("TRAIN:", [train_idx[:10]])
    print(" TEST:", [val_idx[:10]])

TRAIN: [array([ 1,  5,  6,  8, 11, 12, 15, 17, 18, 19])]
 TEST: [array([ 0,  2,  3,  4,  7,  9, 10, 13, 14, 16])]
TRAIN: [array([ 0,  2,  3,  4,  7,  9, 10, 13, 14, 16])]
 TEST: [array([ 1,  5,  6,  8, 11, 12, 15, 17, 18, 19])]


In [461]:
# 전체 label 분포 확인
all_label_count_list = [0 for _ in range(124)]
for i, row in tqdm(label_img_df.iterrows()):
    for box_class in row['box_class_list']:
        bbox_class = int(box_class)
        all_label_count_list[bbox_class] += 1
sum(all_label_count_list) # 전체 bbox 개수

58178it [00:03, 15018.94it/s]


143995

In [462]:
train_label_count_list = [0 for _ in range(124)]
for i, row in tqdm(label_img_df.iterrows()):
    if row['img_id'] not in train_idx:
        continue
    for box_class in row['box_class_list']:
        bbox_class = int(box_class)
        train_label_count_list[bbox_class] += 1
sum(train_label_count_list) # 전체 bbox 개수

58178it [00:05, 10570.19it/s]


72076

In [463]:
val_label_count_list = [0 for _ in range(124)]
for i, row in tqdm(label_img_df.iterrows()):
    if row['img_id'] not in val_idx:
        continue
    for box_class in row['box_class_list']:
        bbox_class = int(box_class)
        val_label_count_list[bbox_class] += 1
sum(val_label_count_list) # 전체 bbox 개수

58178it [00:05, 10554.37it/s]


71919

In [464]:
72076 + 71919 # 143995 나오면 제대로 계산된것

143995

In [465]:
all_label_count_list = [i/143995 for i in all_label_count_list] # 비율로 수정
train_label_count_list = [i/72076 for i in train_label_count_list] # 비율로 수정
val_label_count_list = [i/71919 for i in val_label_count_list] # 비율로 수정

In [466]:
# 기존 train과의 각 class별 분포 차이의 누적을 계산하여 train과 val 중, class 분포 차이의 누적이 더 작은 것을 subset으로 선택
all_train_gap_list = [abs(all_label_count_list[i]-train_label_count_list[i]) for i in range(len(all_label_count_list))]
all_val_gap_list = [abs(all_label_count_list[i]-val_label_count_list[i]) for i in range(len(all_label_count_list))]

In [467]:
# 124개의 각각의 class에서의 분포 차이의 누적이 약 2%정도 인것으로 보아 잘 나누어진듯
print('all과 train과의 class 분포 차이의 누적', sum(all_train_gap_list))
print('all과 val과의 class 분포 차이의 누적', sum(all_val_gap_list))

all과 train과의 class 분포 차이의 누적 0.022688988654118485
all과 val과의 class 분포 차이의 누적 0.022738518975990288


- train의 all과의 분포차이가 조금 더 작으므로 train을 subset으로 선정

### 생성한 subset(all의 절반)을 다시 한번 더 5개의 fold로 multilabel stf kfold 나누어 train과 val을 8:2로 split

In [468]:
subset_index = list(train_idx)
len(subset_index)

29089

In [469]:
subset_df = label_img_df.loc[subset_index]
subset_df.shape

(29089, 7)

In [470]:
## subset의 class 분포 
subset_label_count_list = train_label_count_list.copy()

In [471]:
cv = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=41)
subset_validation_gap_list = [] # 각 index별로, 각 split의 분포 차이 값들어감, idx0=split0
train_validation_gap_list = [] # 각 index별로, 각 split의 분포 차이 값들어감, idx0=split0
for fold_ind, (train_idx, val_idx) in enumerate(cv.split(subset_df, subset_df[['object_num', 'session', 'box_mean_size_class']])):
    
    # class 개수 count
    train_label_count_list = [0 for _ in range(124)]
    for i, row in subset_df.iterrows():
        if row['img_id'] not in train_idx:
            continue
        for box_class in row['box_class_list']:
            bbox_class = int(box_class)
            train_label_count_list[bbox_class] += 1
            
            
    val_label_count_list = [0 for _ in range(124)]
    for i, row in subset_df.iterrows():
        if row['img_id'] not in val_idx:
            continue
        for box_class in row['box_class_list']:
            bbox_class = int(box_class)
            val_label_count_list[bbox_class] += 1
            
    # 비율로 수정
    train_label_count_list = [i/sum(train_label_count_list) for i in train_label_count_list] 
    val_label_count_list = [i/sum(val_label_count_list) for i in val_label_count_list] 
    
    # subset(기존) 과의 class 분포 차이 
    subset_val_gap_list = [abs(subset_label_count_list[i]-val_label_count_list[i]) for i in range(len(val_label_count_list))]
    
    # train과 val과의 class 분포 차이
    train_val_gap_list = [abs(train_label_count_list[i]-val_label_count_list[i]) for i in range(len(val_label_count_list))]

    # split별 분포 차이 값 저장
    subset_validation_gap_list.append(sum(subset_val_gap_list))
    train_validation_gap_list.append(sum(train_val_gap_list))
    
    print("TRAIN:", [train_idx[:10]])
    print(" TEST:", [val_idx[:10]])

TRAIN: [array([ 0,  1,  2,  3,  5,  6,  7,  9, 10, 11])]
 TEST: [array([ 4,  8, 20, 30, 36, 37, 38, 42, 46, 61])]
TRAIN: [array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])]
 TEST: [array([ 0, 13, 17, 21, 23, 27, 28, 32, 35, 44])]
TRAIN: [array([ 0,  1,  3,  4,  5,  6,  7,  8, 10, 13])]
 TEST: [array([ 2,  9, 11, 12, 18, 22, 24, 31, 40, 45])]
TRAIN: [array([ 0,  2,  3,  4,  7,  8,  9, 10, 11, 12])]
 TEST: [array([ 1,  5,  6, 16, 29, 33, 34, 39, 43, 66])]
TRAIN: [array([ 0,  1,  2,  4,  5,  6,  8,  9, 11, 12])]
 TEST: [array([ 3,  7, 10, 14, 15, 19, 25, 26, 41, 47])]


In [472]:
subset_validation_gap_list # subset과의 차이

[0.11314574723857473,
 0.11256823087833619,
 0.10579449894729644,
 0.10125782915684145,
 0.10525734878917632]

In [473]:
train_validation_gap_list # splot 내 train과 val의 차이

[0.13253110411499652,
 0.12883472064656465,
 0.11919501404701233,
 0.12032240678474256,
 0.12680114018890096]

- 기존 subset과의 분포 차이가 가장 작으면서, 나누어진 split내의 train과 val간의 분포차이가 가장 작은 split3(4번째 split) 선택

In [475]:
0.12032240678474256/124 * 100 # 각 class 별로 평균 0.09%정도의 분포차이 -> 잘 나눠진듯

0.09703419901995368

In [476]:
cv = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=41)
for fold_ind, (train_idx, val_idx) in enumerate(cv.split(subset_df, subset_df[['object_num', 'session', 'box_mean_size_class']])):
    
    if fold_ind==3:
        fold3_best_train = train_idx
        fold3_best_val = val_idx
    
print(fold3_best_train[:10])
print(fold3_best_val[:10])
train_idx_list = fold3_best_train
val_idx_list = fold3_best_val
print(f'train length: {len(train_idx_list)}, val length: {len(val_idx_list)}')
# train, val 각각으로 사용할 image index 추출

[ 0  2  3  4  7  8  9 10 11 12]
[ 1  5  6 16 29 33 34 39 43 66]
train length: 23271, val length: 5818


In [477]:
subset_df = subset_df.reset_index()
subset_df = subset_df.reset_index()

In [478]:
subset_df.rename(columns={'level_0':'new_img_id'}, inplace=True)
subset_df.head()

,new_img_id,index,img_id,img_name,object_num,box_mean_size,session,box_class_list,box_mean_size_class
0,0,0,0,10_hik_204_1662356574.mov_280,3,0.001357,10,"[96, 95, 98]",0
1,1,2,2,9_hik_205_1662348299.mov_3328,2,0.004400,9,"[93, 87]",1
2,2,3,3,5_hik_203_1658977700.mov_2295,1,0.010570,5,[49],3
3,3,4,4,4_hik_205_1658973309.mov_165,2,0.003005,4,"[42, 43]",1
4,4,7,7,9_hik_203_1662348669.mov_3288,3,0.001357,9,"[91, 86, 91]",0


In [479]:
multi_train_df = subset_df.loc[list(train_idx_list)]
multi_val_df = subset_df.loc[list(val_idx_list)]

In [480]:
len(multi_train_df), len(multi_val_df)

(23271, 5818)

In [482]:
# box_mean_size_class 잘 나누어졌는지 확인
train_bbox_size_list = multi_train_df.box_mean_size_class.value_counts().sort_index().to_list()
train_bbox_size_ratio = [round(i/23271,3) for i in train_bbox_size_list]

val_bbox_size_list = multi_val_df.box_mean_size_class.value_counts().sort_index().to_list()
val_bbox_size_ratio = [round(i/5818,3) for i in val_bbox_size_list] 

print('train',train_bbox_size_ratio,'\n')
print('val',val_bbox_size_ratio)

train [0.25, 0.252, 0.247, 0.251] 

val [0.25, 0.243, 0.255, 0.252]


In [483]:
# object_num이 잘 나누어졌는지 확인
train_object_num_class_list = multi_train_df.object_num.value_counts().sort_index().to_list()
train_object_num_class_ratio = [round(i/23271,3) for i in train_object_num_class_list]

val_object_num_list = multi_val_df.object_num.value_counts().sort_index().to_list()
val_object_num_ratio = [round(i/5818,3) for i in val_object_num_list] 

print('train',train_object_num_class_ratio,'\n')
print('val',val_object_num_ratio)

train [0.239, 0.281, 0.293, 0.152, 0.023, 0.007, 0.003, 0.001, 0.0] 

val [0.229, 0.299, 0.287, 0.152, 0.022, 0.006, 0.003, 0.001, 0.0]


In [484]:
# session가 잘 나누어졌는지 확인
train_session_list = multi_train_df.session.value_counts().sort_index().to_list()
train_session_ratio = [round(i/23271,3) for i in train_session_list]

val_session_list = multi_val_df.session.value_counts().sort_index().to_list()
val_session_ratio = [round(i/5818,3) for i in val_session_list] 

print('train',train_session_ratio,'\n')
print('val',val_session_ratio)

train [0.098, 0.071, 0.067, 0.07, 0.07, 0.063, 0.069, 0.065, 0.073, 0.108, 0.101, 0.07, 0.076] 

val [0.098, 0.076, 0.071, 0.065, 0.074, 0.06, 0.07, 0.061, 0.065, 0.107, 0.105, 0.069, 0.079]


- 모두 고르게 잘 나누어졌으므로 txt파일 생성

In [490]:
# label이 없는 이미지도 FP 줄이기 위해 비율(10%) 유지하여 추가(합쳐진 train data의 10% : 약 2585개)
len(empty_txt_img_list), len(no_label_img_list) # empty_txt_img_list만 이용하는게 나으려나

(675, 7596)

In [519]:
empty_txt_img_list[0]

'7_hik_205_1658988715.mov_345.jpg'

In [491]:
no_label_img_list.extend(empty_txt_img_list)

In [492]:
len(no_label_img_list)

8271

In [493]:
# random하게 섞은 후 리스트 앞 3222개 추출하여 추가
import random
random.seed(41)
random.shuffle(no_label_img_list)
no_label_img_list[:10]

['4_hik_203_1658972857.mov_4650.jpg',
 '3rd Run_1_4968_hanwha_249.mp4_8880.jpg',
 '11_hik_205_1662359355.mov_1144.jpg',
 '3rd Run_1_4969_hik_246.mp4_1170.jpg',
 '11_hik_205_1662360339.mov_2388.jpg',
 '10_hik_204_1662357506.mov_24.jpg',
 '9_hik_205_1662354443.mov_2628.jpg',
 '7_hik_203_1658989795.mov_1320.jpg',
 '3rd Run_1_4974_hik_246.mp4_2700.jpg',
 '5_hik_203_1658977700.mov_1395.jpg']

In [518]:
for i in no_label_img_list:
    
    if i == '3_hik_203_1658888182.mov_15.jpg':
        print(i)

3_hik_203_1658888182.mov_15.jpg


In [494]:
with open('/opt/ml/pkt_train_124.txt', 'w') as f:
    for i, row in multi_train_df.iterrows():
        img_name = row['img_name']
        img_path = 'dataset/images/' + img_name + '.jpg\n'
        f.write(img_path)
    for j in range(2585):
        no_label_img_path = 'dataset/images/' + no_label_img_list[j] + '\n'
        f.write(no_label_img_path)

In [495]:
with open('/opt/ml/nolableimage_used_in_pkt_train_124.txt', 'w') as f:
    for j in range(2585):
        no_label_img_path = 'dataset/images/' + no_label_img_list[j] + '\n'
        f.write(no_label_img_path)

In [496]:
with open('/opt/ml/pkt_val_124.txt', 'w') as f:
    for i, row in multi_val_df.iterrows():
        img_name = row['img_name']
        img_path = 'dataset/images/' + img_name + '.jpg\n'
        f.write(img_path)

In [498]:
23271+2585

25856